## Chapter 7. Aggregations

In [105]:
df.show(2)
spark.sql("select * from dfTable").show(2)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 2 rows

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|      

In [106]:
df.select(count("StockCode")).show()

+----------------+
|count(StockCode)|
+----------------+
|          541909|
+----------------+



In [107]:
import org.apache.spark.sql.functions.count
spark.sql("SELECT COUNT(*) FROM dfTable").show()

+--------+
|count(1)|
+--------+
|  541909|
+--------+



In [108]:
import org.apache.spark.sql.functions.countDistinct
df.select(countDistinct("StockCode")).show()

+-------------------------+
|count(DISTINCT StockCode)|
+-------------------------+
|                     4070|
+-------------------------+



In [109]:
import org.apache.spark.sql.functions.approx_count_distinct
df.select(approx_count_distinct("StockCode", 0.1)).show() // 3364

+--------------------------------+
|approx_count_distinct(StockCode)|
+--------------------------------+
|                            3364|
+--------------------------------+



In [110]:
import org.apache.spark.sql.functions.{first, last}
df.select(first("StockCode"), last("StockCode")).show()

+-----------------------+----------------------+
|first(StockCode, false)|last(StockCode, false)|
+-----------------------+----------------------+
|                 85123A|                 22138|
+-----------------------+----------------------+



In [111]:
import org.apache.spark.sql.functions.{min, max}
df.select(min("Quantity"), max("Quantity")).show()

+-------------+-------------+
|min(Quantity)|max(Quantity)|
+-------------+-------------+
|       -80995|        80995|
+-------------+-------------+



In [112]:
import org.apache.spark.sql.functions.sum
df.select(sum("Quantity")).show()

+-------------+
|sum(Quantity)|
+-------------+
|      5176450|
+-------------+



In [113]:
import org.apache.spark.sql.functions.sumDistinct
df.select(sumDistinct("Quantity")).show()

+----------------------+
|sum(DISTINCT Quantity)|
+----------------------+
|                 29310|
+----------------------+



In [114]:
import org.apache.spark.sql.functions.{sum, count, avg, expr}
df.select(
count("Quantity").alias("total_transactions"),
sum("Quantity").alias("total_purchases"),
avg("Quantity").alias("avg_purchases"),
expr("mean(Quantity)").alias("mean_purchases"))
.selectExpr(
"total_purchases/total_transactions as DerivedAVG",
"avg_purchases",
"mean_purchases").show()

+----------------+----------------+----------------+
|      DerivedAVG|   avg_purchases|  mean_purchases|
+----------------+----------------+----------------+
|9.55224954743324|9.55224954743324|9.55224954743324|
+----------------+----------------+----------------+



In [115]:
import org.apache.spark.sql.functions.{var_pop, stddev_pop}
import org.apache.spark.sql.functions.{var_samp, stddev_samp}
df.select(var_pop("Quantity"), var_samp("Quantity"),stddev_pop("Quantity"), stddev_samp("Quantity")).show()

+------------------+------------------+--------------------+---------------------+
| var_pop(Quantity)|var_samp(Quantity)|stddev_pop(Quantity)|stddev_samp(Quantity)|
+------------------+------------------+--------------------+---------------------+
|47559.303646609354| 47559.39140929905|  218.08095663447864|   218.08115785023486|
+------------------+------------------+--------------------+---------------------+



In [116]:
import org.apache.spark.sql.functions.{skewness, kurtosis}
df.select(skewness("Quantity"), kurtosis("Quantity")).show()

+--------------------+------------------+
|  skewness(Quantity)|kurtosis(Quantity)|
+--------------------+------------------+
|-0.26407557610527843|119768.05495536518|
+--------------------+------------------+



In [117]:
import org.apache.spark.sql.functions.{corr, covar_pop, covar_samp}
df.select(corr("InvoiceNo", "Quantity"), covar_samp("InvoiceNo", "Quantity"),covar_pop("InvoiceNo", "Quantity")).show()

+-------------------------+-------------------------------+------------------------------+
|corr(InvoiceNo, Quantity)|covar_samp(InvoiceNo, Quantity)|covar_pop(InvoiceNo, Quantity)|
+-------------------------+-------------------------------+------------------------------+
|     4.912186085637639E-4|             1052.7280543913773|            1052.7260778752732|
+-------------------------+-------------------------------+------------------------------+



In [118]:
import org.apache.spark.sql.functions.{collect_set, collect_list}
df.agg(collect_set("Country"), collect_list("Country")).show()

+--------------------+---------------------+
|collect_set(Country)|collect_list(Country)|
+--------------------+---------------------+
|[Portugal, Italy,...| [United Kingdom, ...|
+--------------------+---------------------+



In [119]:
df.groupBy("InvoiceNo", "CustomerId").count().show(2)

+---------+----------+-----+
|InvoiceNo|CustomerId|count|
+---------+----------+-----+
|   536846|     14573|   76|
|   537026|     12395|   12|
+---------+----------+-----+
only showing top 2 rows



In [120]:
import org.apache.spark.sql.functions.count
df.groupBy("InvoiceNo").agg(
count("Quantity").alias("quan"),
expr("count(Quantity)")).show(2)

+---------+----+---------------+
|InvoiceNo|quan|count(Quantity)|
+---------+----+---------------+
|   536596|   6|              6|
|   536938|  14|             14|
+---------+----+---------------+
only showing top 2 rows



In [121]:
//transformations as a series of Maps for which the key is the column, and the value is the aggregation function 
//(as a string) that you would like to perform.Here avg/stddev_pop are the map function
df.groupBy("InvoiceNo").agg("Quantity"->"avg").show(2)
df.groupBy("InvoiceNo").agg("Quantity"->"avg", "Quantity"->"sum", "Quantity"->"count","Quantity"->"stddev_pop").show(2)

+---------+------------------+
|InvoiceNo|     avg(Quantity)|
+---------+------------------+
|   536596|               1.5|
|   536938|33.142857142857146|
+---------+------------------+
only showing top 2 rows

+---------+------------------+-------------+---------------+--------------------+
|InvoiceNo|     avg(Quantity)|sum(Quantity)|count(Quantity)|stddev_pop(Quantity)|
+---------+------------------+-------------+---------------+--------------------+
|   536596|               1.5|            9|              6|  1.1180339887498947|
|   536938|33.142857142857146|          464|             14|  20.698023172885524|
+---------+------------------+-------------+---------------+--------------------+
only showing top 2 rows



#### Window Functions

In [122]:
// Add a date columnfrom a String column(InvoiceDate)
import org.apache.spark.sql.functions.{col, to_date}
val dfWithDate = df.withColumn("date", to_date(col("InvoiceDate"),"MM/d/yyyy H:mm"))
dfWithDate.printSchema()
dfWithDate.createOrReplaceTempView("dfWithDate")

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- date: date (nullable = true)



dfWithDate = [InvoiceNo: string, StockCode: string ... 7 more fields]


[InvoiceNo: string, StockCode: string ... 7 more fields]

In [123]:
//Create a window 
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions.col
val windowSpec = Window
.partitionBy("CustomerId", "date")
.orderBy(col("Quantity").desc)
.rowsBetween(Window.unboundedPreceding, Window.currentRow)

windowSpec = org.apache.spark.sql.expressions.WindowSpec@696ec602


org.apache.spark.sql.expressions.WindowSpec@696ec602

In [124]:
// get max  for the window 
import org.apache.spark.sql.functions.max
val maxPurchaseQuantity = max(col("Quantity")).over(windowSpec)

maxPurchaseQuantity = max(Quantity) OVER (PARTITION BY CustomerId, date ORDER BY Quantity DESC NULLS LAST ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)


max(Quantity) OVER (PARTITION BY CustomerId, date ORDER BY Quantity DESC NULLS LAST ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)

In [ ]:
//create rank variables for the window 
import org.apache.spark.sql.functions.{dense_rank, rank}
val purchaseDenseRank = dense_rank().over(windowSpec)
val purchaseRank = rank().over(windowSpec)

In [126]:
import org.apache.spark.sql.functions.col
dfWithDate.where("CustomerId IS NOT NULL").orderBy("CustomerId")
.select(
col("CustomerId"),
col("date"),
col("Quantity"),
purchaseRank.alias("quantityRank"),
purchaseDenseRank.alias("quantityDenseRank"),
maxPurchaseQuantity.alias("maxPurchaseQuantity")).show(5)

+----------+----------+--------+------------+-----------------+-------------------+
|CustomerId|      date|Quantity|quantityRank|quantityDenseRank|maxPurchaseQuantity|
+----------+----------+--------+------------+-----------------+-------------------+
|     12346|2011-01-18|   74215|           1|                1|              74215|
|     12346|2011-01-18|  -74215|           2|                2|              74215|
|     12347|2010-12-07|      36|           1|                1|                 36|
|     12347|2010-12-07|      30|           2|                2|                 36|
|     12347|2010-12-07|      24|           3|                3|                 36|
+----------+----------+--------+------------+-----------------+-------------------+
only showing top 5 rows



#### Grouping Sets

In [127]:
// drop is used for removing null values, check chapter 6
val dfNoNull = dfWithDate.drop()
dfNoNull.createOrReplaceTempView("dfNoNull")

dfNoNull = [InvoiceNo: string, StockCode: string ... 7 more fields]


[InvoiceNo: string, StockCode: string ... 7 more fields]

In [128]:
spark.sql("""SELECT CustomerId, stockCode, sum(Quantity) FROM dfNoNull 
GROUP BY customerId, stockCode
ORDER BY CustomerId DESC, stockCode DESC""").show(2)

+----------+---------+-------------+
|CustomerId|stockCode|sum(Quantity)|
+----------+---------+-------------+
|     18287|    85173|           48|
|     18287|   85040A|           48|
+----------+---------+-------------+
only showing top 2 rows



#### Rollups

In [141]:
//groupby total, sub total, Grand Total ( but sub total only on one of the parameter)
dfNoNull.rollup("Date", "Country").agg(sum("Quantity"))
.selectExpr("Date", "Country", "`sum(Quantity)` as total_quantity").orderBy("Date","Country").show(10)

+----------+--------------+--------------+
|      Date|       Country|total_quantity|
+----------+--------------+--------------+
|      null|          null|       5176450|
|2010-12-01|          null|         26814|
|2010-12-01|     Australia|           107|
|2010-12-01|          EIRE|           243|
|2010-12-01|        France|           449|
|2010-12-01|       Germany|           117|
|2010-12-01|   Netherlands|            97|
|2010-12-01|        Norway|          1852|
|2010-12-01|United Kingdom|         23949|
|2010-12-02|          null|         21023|
+----------+--------------+--------------+
only showing top 10 rows



#### Cube

In [130]:
////groupby total, sub total, Grand Total ( ub total only on both of the parameter)
dfNoNull.cube("Date", "Country")
.agg(sum(col("Quantity")))
.select("Date", "Country", "sum(Quantity)").orderBy("Date", "Country").show(10)

+----+---------------+-------------+
|Date|        Country|sum(Quantity)|
+----+---------------+-------------+
|null|           null|      5176450|
|null|      Australia|        83653|
|null|        Austria|         4827|
|null|        Bahrain|          260|
|null|        Belgium|        23152|
|null|         Brazil|          356|
|null|         Canada|         2763|
|null|Channel Islands|         9479|
|null|         Cyprus|         6317|
|null| Czech Republic|          592|
+----+---------------+-------------+
only showing top 10 rows



lastException: Throwable = null


#### Grouping Metadata

In [131]:
//3  total quantity regardless of customerId and stockCode.
//2 This will appear for all aggregations of individual stock codes.
//1 This will give us the total quantity on a per-customer basis.
//0 This will give us the total quantity for individual customerId and stockCode combinations.
import org.apache.spark.sql.functions.{grouping_id, sum, expr}
dfNoNull.cube("customerId", "stockCode")
.agg(grouping_id(), sum("Quantity"))
.orderBy(expr("grouping_id()").desc).show(10)

+----------+---------+-------------+-------------+
|customerId|stockCode|grouping_id()|sum(Quantity)|
+----------+---------+-------------+-------------+
|      null|     null|            3|      5176450|
|      null|   47590B|            2|         2244|
|      null|    22275|            2|           69|
|      null|   90059E|            2|           19|
|      null|    22295|            2|         2795|
|      null|    22919|            2|         1745|
|      null|    22207|            2|         1259|
|      null|    22265|            2|          540|
|      null|    84670|            2|           23|
|      null|   51014C|            2|         2505|
+----------+---------+-------------+-------------+
only showing top 10 rows



#### Pivot

In [ ]:
val pivoted = dfWithDate.groupBy("date").pivot("Country").sum()

In [133]:
pivoted.where("date > '2011-12-05'").select("date" ,"`USA_sum(Quantity)`").show()

+----------+-----------------+
|      date|USA_sum(Quantity)|
+----------+-----------------+
|2011-12-06|             null|
|2011-12-09|             null|
|2011-12-08|             -196|
|2011-12-07|             null|
+----------+-----------------+



#### UDAF

In [134]:
import org.apache.spark.sql.expressions.MutableAggregationBuffer
import org.apache.spark.sql.expressions.UserDefinedAggregateFunction
import org.apache.spark.sql.Row
import org.apache.spark.sql.types._
class BoolAnd extends UserDefinedAggregateFunction {
def inputSchema: org.apache.spark.sql.types.StructType =
StructType(StructField("value", BooleanType) :: Nil)
def bufferSchema: StructType = StructType(
StructField("result", BooleanType) :: Nil
)
def dataType: DataType = BooleanType
def deterministic: Boolean = true
def initialize(buffer: MutableAggregationBuffer): Unit = {
buffer(0) = true
}
def update(buffer: MutableAggregationBuffer, input: Row): Unit = {
buffer(0) = buffer.getAs[Boolean](0) && input.getAs[Boolean](0)
}
def merge(buffer1: MutableAggregationBuffer, buffer2: Row): Unit = {
buffer1(0) = buffer1.getAs[Boolean](0) && buffer2.getAs[Boolean](0)
}
def evaluate(buffer: Row): Any = {
buffer(0)
}
}

defined class BoolAnd


In [ ]:
val ba = new BoolAnd //instantiate our class
spark.udf.register("booland", ba)// or register it as a function

In [142]:
import org.apache.spark.sql.functions._
spark.range(1)
.selectExpr("explode(array(TRUE, TRUE, TRUE)) as t")
.selectExpr("explode(array(TRUE, FALSE, TRUE)) as f", "t")
.select(ba(col("t")), expr("booland(f)"))
.show()

+----------+----------+
|booland(t)|booland(f)|
+----------+----------+
|      true|     false|
+----------+----------+

